In [ ]:
%pip install setuptools==65.5.0 pip==21
%pip install wheel==0.38.0
%pip install flappy-bird-gym

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      c:\Users\zhangj20\Documents\code_v1.1\myenv\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      error in gym setup command: 'extras_require' must be a dictionary whose values are strings or lists of strings containing valid project/version requirement specifiers.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> gym

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.



  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


In [3]:
# import gym
import flappy_bird_gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# Parameters
HISTORY_SIZE = 4  # Tunable parameter: number of past states to include as features
LEARNING_RATE = 1e-3
GAMMA = 0.99
BATCH_SIZE = 32
EPSILON_DECAY = 0.995
MIN_EPSILON = 0.01
MEMORY_SIZE = 10000

ModuleNotFoundError: No module named 'flappy_bird_gym'

In [ ]:


# Neural Network for Q-learning
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Replay memory to store past experiences
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, transition):
        self.memory.append(transition)

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

# Deep Q-learning agent
class DQNAgent:
    def __init__(self, action_size, state_size, history_size):
        self.action_size = action_size
        self.history_size = history_size
        self.epsilon = 1.0  # Start with high exploration
        self.memory = ReplayMemory(MEMORY_SIZE)
        self.model = QNetwork(state_size * history_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.action_size)  # Explore
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            return self.model(state).argmax().item()  # Exploit

    def store_experience(self, experience):
        self.memory.push(experience)

    def train_step(self):
        if len(self.memory) < BATCH_SIZE:
            return

        batch = self.memory.sample(BATCH_SIZE)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions).unsqueeze(1)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)

        current_q = self.model(states).gather(1, actions).squeeze()
        max_next_q = self.model(next_states).max(1)[0]
        expected_q = rewards + (GAMMA * max_next_q * (1 - dones))

        loss = nn.functional.mse_loss(current_q, expected_q.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_epsilon(self):
        if self.epsilon > MIN_EPSILON:
            self.epsilon *= EPSILON_DECAY

# Training loop
def train(agent, episodes):
    env = flappy_bird_gym.make("FlappyBird-v0")
    for episode in range(episodes):
        state = env.reset()
        state_history = deque([state] * agent.history_size, maxlen=agent.history_size)
        total_reward = 0

        while True:
            state_input = np.array(state_history).flatten()
            action = agent.select_action(state_input)
            next_state, reward, done, _ = env.step(action)

            next_state_history = list(state_history)
            next_state_history.append(next_state)
            next_state_input = np.array(next_state_history).flatten()

            agent.store_experience((state_input, action, reward, next_state_input, done))
            agent.train_step()
            state_history.append(next_state)
            total_reward += reward

            if done:
                agent.update_epsilon()
                print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")
                break

    env.close()

# Initialize and train the agent
state_size = 2  # state space size for flappy bird
action_size = 2  # action space (flap or no flap)
agent = DQNAgent(action_size, state_size, HISTORY_SIZE)

# Train for 1000 episodes
train(agent, 1000)


ModuleNotFoundError: No module named 'distutils'